In [89]:
import os
import sys
import platform
import pandas as pd
import re
from datetime import datetime, timedelta

In [90]:
import requests
from bs4 import BeautifulSoup
import html2text
from datetime import datetime

In [91]:
def get_minutes_list(from_date='20050101'):
    prefix_addr = "https://www.bok.or.kr"
    from_date = datetime.strptime(from_date, '%Y%m%d')

    for pageIndex in range(1, 31):
        url = 'https://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex={}'.format(pageIndex)
        user_agent = 'Mozilla/5.0'
        headers ={'User-Agent' : user_agent}
        page = requests.get(url, headers=headers)

        try:
            soup = BeautifulSoup(page.content, 'html.parser')
            brdList = soup.find_all('span', class_='col m10 s10 x9 ctBx')
            datainfo = soup.find_all('div', class_='col s12 dataInfo')

            for post, data in zip(brdList, datainfo):
                guid = prefix_addr+post.a['href'][:-12]

                desPage = requests.get(guid)
                desSoup = BeautifulSoup(desPage.content, 'html.parser')   
                description = desSoup.find('div', class_='dbData').get_text().strip()
                if description.replace(' ','').find('통화정책방향') >= 0:
                    title = post.find('span', class_='titlesub').get_text().strip()
        
                    mdate = title[title.find(')(') + 2:-1]            
                    if mdate[-1] == '.':
                        mdate = mdate[:-1]
                    mdate = datetime.strptime(mdate, '%Y.%m.%d')

                    if mdate < from_date:
                        break

                    rdate = data.find('span', class_='date').get_text().strip()
                    rdate = datetime.strptime(rdate[3:], '%Y.%m.%d')    

                    get_minutes_file(guid, mdate, rdate)
        except:
            print("get url.content error and pass page{} it".format(pageIndex))

In [92]:
def get_minutes_file(page_addr, mdate, rdate):
    file_header = 'data/minutes/pdf/'
    prefix_addr = "http://bok.or.kr"

    page = requests.get(page_addr)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    try:
        links = soup.find('div', class_='addfile').find_all('a')

        for link in links:
            filename = link.get_text()
            filename = filename.replace('\r', '').replace('\t', '').replace('\n', '')

            if filename[-3:] == 'pdf':
                filename = mdate.strftime('%Y%m%d') + "_" + rdate.strftime('%Y%m%d')+ '.pdf'
                file_addr = prefix_addr + link["href"]
                file_res = requests.get(file_addr)
                filepath = file_header + filename

                with open(filepath, 'wb') as f:
                    f.write(file_res.content)

                print('save file name : ' + filename)
    except:
        print("get file failed and pass it")

In [93]:
get_minutes_list()

save file name : 20190418_20190507.pdf
save file name : 20190228_20190319.pdf
save file name : 20190124_20190212.pdf
save file name : 20181130_20181218.pdf
save file name : 20181018_20181106.pdf
save file name : 20180831_20180918.pdf
save file name : 20180712_20180731.pdf
save file name : 20180524_20180612.pdf
save file name : 20180412_20180502.pdf
save file name : 20180227_20180320.pdf
save file name : 20180118_20180206.pdf
save file name : 20171130_20171219.pdf
save file name : 20170831_20170919.pdf
save file name : 20170713_20170801.pdf
save file name : 20170525_20170613.pdf
save file name : 20170413_20170502.pdf
save file name : 20170223_20170314.pdf
save file name : 20170113_20170131.pdf
save file name : 20161215_20170103.pdf
save file name : 20161111_20161129.pdf
save file name : 20161013_20161101.pdf
save file name : 20160909_20160927.pdf
save file name : 20160811_20160830.pdf
save file name : 20160714_20160802.pdf
save file name : 20160609_20160628.pdf
save file name : 20160513

In [94]:
import re
from tika import parser
from os import listdir
from os.path import isfile, join

# 폴더내 pdf파일을 txt 파일로 변경
def pdf2txt (source_folder="./data/minutes/pdf/", output_folder="./data/minutes/text/") :
    # 지정 폴더 내 파일 목록 조회 (파일만)
    pdf_files = [f for f in listdir(source_folder) if isfile(join(source_folder, f))]
    
    for pdf in pdf_files :
        pdf_filepath = source_folder + pdf
        # pdf 파일을 text로 변환
        parsedPDF = re.sub('\n', '', parser.from_file(pdf_filepath)["content"])
        
        with open((output_folder + pdf).replace('.pdf', '.txt'), 'w') as f :
            f.write(parsedPDF)
            f.close()      

In [95]:
pdf2txt()

In [96]:
import csv
import pandas as pd
from os import listdir
from os.path import isfile, join

def txt2csv(source_folder="./data/minutes/text/", output_file="./data/minutes/minutes.csv") :
    # 지정 폴더 내 파일 목록 조회 (파일만)
    txt_files = [f for f in listdir(source_folder) if isfile(join(source_folder, f))]
    df = pd.DataFrame(columns=['date', 'minutes'])
    
    for txt_file in txt_files :
        with open(source_folder + txt_file, 'r') as f :
            txt = f.read()
            df.loc[len(df)] = [txt_file.split('_')[0], txt]
    
    df.to_csv(output_file, index=False)
        
    print(df.shape)

In [97]:
#http://openuiz.blogspot.com/2016/07/mecab-ko-dic.html
import pandas as pd
from ekonlpy.tag import Mecab

def Preprocessing(input_file="./data/minutes/minutes.csv", output_file="./data/minutes/minutes_preprocessed.csv") :
    mecab = Mecab()
    df = pd.read_csv(input_file)
    df['preprocessed'] = ''

    #return 
    for row_idx in df.index: 
        text = df['minutes'][row_idx]
        
        try : 
            token_list = mecab.pos(text)
            stoppos = ['SC','SY','SF','SE','SS','SP','SO','SW','SSC','JKS','JKC','JKG','JKO','JKB','JKV','JKQ','JX','JC','EF','EC','ETN','ETM']
            stopword = ['']

            word = []
            for tag in token_list:
                if tag[1] not in stoppos:
                    if tag[0] not in stopword:
                        word.append(tag[0])

            #print(word)
            #print(",".join(word))
            df['preprocessed'][row_idx] = ",".join(word)
            print(str(row_idx+1) +'/'+ str(df.shape[0]) +'번째 파일 전처리 완료')
            #print( df['preprocessed'][row_idx] )
        except:
            print("Error :" + text)
        
    df.to_csv(output_file, index=False)
        
Preprocessing()

/usr/lib/python3/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1/158번째 파일 전처리 완료
2/158번째 파일 전처리 완료
3/158번째 파일 전처리 완료
4/158번째 파일 전처리 완료
5/158번째 파일 전처리 완료
6/158번째 파일 전처리 완료
7/158번째 파일 전처리 완료
8/158번째 파일 전처리 완료
9/158번째 파일 전처리 완료
10/158번째 파일 전처리 완료
11/158번째 파일 전처리 완료
12/158번째 파일 전처리 완료
13/158번째 파일 전처리 완료
14/158번째 파일 전처리 완료
15/158번째 파일 전처리 완료
16/158번째 파일 전처리 완료
17/158번째 파일 전처리 완료
18/158번째 파일 전처리 완료
19/158번째 파일 전처리 완료
20/158번째 파일 전처리 완료
21/158번째 파일 전처리 완료
22/158번째 파일 전처리 완료
23/158번째 파일 전처리 완료
24/158번째 파일 전처리 완료
25/158번째 파일 전처리 완료
26/158번째 파일 전처리 완료
27/158번째 파일 전처리 완료
28/158번째 파일 전처리 완료
29/158번째 파일 전처리 완료
30/158번째 파일 전처리 완료
31/158번째 파일 전처리 완료
32/158번째 파일 전처리 완료
33/158번째 파일 전처리 완료
34/158번째 파일 전처리 완료
35/158번째 파일 전처리 완료
36/158번째 파일 전처리 완료
37/158번째 파일 전처리 완료
38/158번째 파일 전처리 완료
39/158번째 파일 전처리 완료
40/158번째 파일 전처리 완료
41/158번째 파일 전처리 완료
42/158번째 파일 전처리 완료
43/158번째 파일 전처리 완료
44/158번째 파일 전처리 완료
45/158번째 파일 전처리 완료
46/158번째 파일 전처리 완료
47/158번째 파일 전처리 완료
48/158번째 파일 전처리 완료
49/158번째 파일 전처리 완료
50/158번째 파일 전처리 완료
51/158번째 파일 전처리 완료
52/158번째 파일 전처리 완료
53/158번째 파일 전처리 완료
54

In [4]:
def tidy_sentences(section):
    sentence_enders = re.compile(r'((?<=[함음됨임봄짐움])(\s*\n|\.|;)|(?<=다)\.)\s*')
    splits = list((m.start(), m.end()) for m in re.finditer(sentence_enders, section))
    starts = [0] + [i[1] for i in splits]
    ends = [i[0] for i in splits]
    sentences = [section[start:end] for start, end in zip(starts[:-1], ends)]
    for i, s in enumerate(sentences):
        sentences[i] = (s.replace('\n', ' ').replace(' ', ' ')) + '.'

    text = '\n'.join(sentences) if len(sentences) > 0 else ''
    return sentences, text

In [5]:
import re 

def preprocess_minutes(minutes):

    pos = re.search('(.?국내외\s?경제\s?동향.?과 관련하여,?|\(가\).+경제전망.*|\(가\) 국내외 경제동향 및 평가)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s1 = pos.start() if pos else -1
    # pos = re.search('(.?외환.?국제금융\s?동향.?과 관련하여,?|\(나\) 외환.국제금융\s?(및 금융시장)?\s?동향)\n?\s*일부 위원은', minutes, re.MULTILINE)
    # pos = re.search('(.?외환.?국제금융\s?동향.?과 관련하여.*|\(나\) 외환.국제금융\s?(및 금융시장)?\s?동향)\n?\s*일부 위원은', minutes, re.MULTILINE)
    pos = re.search('(.?외환.?국제금융\s?동향.?과 관련하여.*|\(나\) 외환.국제금융\s?(및 금융시장)?\s?동향)\n?\s*(일부 위원은|대부분의 위원들은)', minutes,
                    re.MULTILINE)
    s2 = pos.start() if pos else -1
    pos = re.search('(.?금융시장\s?동향.?과 관련하여,?|\(다\) 금융시장\s?동향)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s3 = pos.start() if pos else -1
    # pos = re.search('((\((다|라)\) )?.?통화정책방향.?에 관한 토론,?|이상과 같은 의견교환을 바탕으로.*통화정책방향.*에 관해 다음과 같은 토론이 있었음.*)\n?', minutes, re.MULTILINE)
    # pos = re.search('((\((다|라)\) )?.?통화정책.?방향.?에 관한 토론,?|이상과 같은 의견교환을 바탕으로.*통화정책방향.*에.*토론.*)\n?', minutes, re.MULTILINE)
    # pos = re.search('((\((다|라)\) )?.?통화정책\s?방향.?에 관한 토론,?|이상과 같은 의견교환을 바탕으로.*통화정책\s?방향.*에.*토론.*)\n?', minutes, re.MULTILINE)
    pos = re.search('((\((다|라)\) )?.?통화정책\s?방향.?에 관한 토론,?|이상과 같은 의견\s?교환을 바탕으로.*통화정책\s?방향.*에.*토론.*)\n?', minutes,
                    re.MULTILINE)
    s4 = pos.start() if pos else -1
    pos = re.search('(\(4\) 정부측 열석자 발언.*)\n?', minutes, re.MULTILINE)
    s5 = pos.start() if pos else -1
    pos = re.search('(\(.*\) 한국은행 기준금리 결정에 관한 위원별 의견\s?개진|이상과 같은 토론에 이어 .* 관한 위원별 의견개진이 있었음.*)\n?', minutes,
                    re.MULTILINE)
    s6 = pos.start() if pos else -1
    # pos = re.search('(\(\s?.*\s?\) ()(심의결과|토의결론))\n?', minutes, re.MULTILINE)
    # s7 = pos.start() if pos else -1
    positer = re.finditer('(\(\s?.*\s?\) ()(심의결과|토의결론))\n?', minutes, re.MULTILINE)
    s7 = [pos.start() for pos in positer if pos.start() > s6]
    s7 = s7[0] if s7 else -1

    # 국내외 경제동향
    bos = s1
    eos = s2
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section1, section1_txt = tidy_sentences(section)

    # 외환․국제금융 동향
    bos = s2
    eos = s3 if s3 >= 0 else s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section2, section2_txt = tidy_sentences(section)

    # 금융시장 동향
    bos = s3
    eos = s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section3, section3_txt = tidy_sentences(section)

    # 통화정책방향
    bos = s4
    eos = s5 if s5 >= 0 else s6 if s6 >= 0 else s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section4, section4_txt = tidy_sentences(section)

    # 위원별 의견 개진
    bos = s6
    eos = s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section5, section5_txt = tidy_sentences(section)

    # 정부측 열석자 발언
    bos = s5
    eos = s6
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('정부측 열석자 발언', section, re.MULTILINE)
    bos = pos.end() + 1 if pos else -1
    section = section[bos:] if bos >= 0 else section
    section6, section6_txt = tidy_sentences(section)

    sections = ['Economic Situation', 'Foreign Currency', 'Financial Markets',
                'Monetary Policy', 'Participants’ Views', 'Government’s View']
    section_texts = (section1, section2, section3, section4, section5, section6)


    return sections, section_texts

In [100]:
a, b = preprocess_minutes(df['minutes'][0])

KeyError: 'minutes'

In [3]:
import pandas as pd
from ekonlpy.sentiment import MPCK

def text2ngram(text) :
    mpck = MPCK()
    bef_tokens = mpck.tokenize(text)
    ngrams = mpck.ngramize(bef_tokens)
    tokens = []

    #print(tokens)
    #print(ngrams)
    #break;

    stoppos = ['SC','SY','SF','SE','SS','SP','SO','SW','SSC','JKS','JKC','JKG','JKO','JKB','JKV','JKQ','JX','JC','EF','EC','ETN','ETM']
    stopword = ['']

    for tag in bef_tokens:
        if tag[1] not in stoppos:
            if tag[0] not in stopword:
                tokens.append(tag)
                
    return tokens, ngrams

In [31]:
' '.join(b[4])

'일부 위원은 다음과 같은 점을 종합적으로 고려할 때 이번 달에는 콜금리 - 6 -목표를 0.25% 포인트 인상하는 것이 바람직하다고 생각된다는 의견을 표명하였음   첫째, 실물경제지표는 우리 경제가 완만한 상승세에 있음을 보여주고 있음. 소비와 설비투자가 꾸준한 증가세를 지속하고 수출도 두 자릿수의 증가율을 이어가고 있으며 건설투자가 예상보다 부진한 모습이나 하반기에는 토목건설을 중심으로 점차 나아질 것으로 전망됨. 다만 중동정세 불안에 따른 국제유가 상승, 대형사업장의 노사분규, 집중호우 피해와 이러한 일련의 사태에서 비롯된 경제주체의 심리 악화 등으로 경제성장의 하방위험이 다소 높아졌다고 생각됨. 이러한 위험요인들이 예상외로 크게 확대될 가능성을 배제할 수 없겠으나 제반 사정을 고려할 때 우리 경제는 완만한 경기감속 후 재반등하는 경로를 밟을 것으로 전망됨. 둘째, 물가는 근원인플레이션율과 소비자물가 상승률 모두 안정세를 유지하고 있어 물가안정목표를 벗어날 위험은 낮다고 보겠으나 고유가 지속으로 인한 물가상승압력은 여전히 높은 상황임. 셋째, 금융시장에서는 가계대출을 중심으로 금융기관의 대출태도가 이전보다 신중해지면서 여신 증가세가 둔화되고 있으나 유동성은 여전히 풍부한 것으로 판단됨   다른 일부 위원은 다음과 같은 점들을 종합적으로 감안할 때 경기의 하방위험이 물가의 상방위험보다 크다고 보이므로 위험관리적 측면에서 통화정책을 접근한다는 관점에서 그동안의 네 차례 정책금리 인상이 길고 가변적인 시차를 두고 물가 및 경제성장에 영향을 미친다는 점을 고려하여 다음 통화정책방향 결정시까지 콜금리 목표를 현 수준인 4.25%에서 유지하는 방향으로 통화정책을 운용하는 것이 바람직하다고 판단된다는 의견을 표명하였음   첫째, 국내 경기동향을 보면, 소비 증가세 둔화와 건설투자 부진으로 2/4분기 GDP성장률이 전기대비 0.8%로 하락하였으며 고용사정 개선도 지연되고 있음. 제조업생산과 서비스업활동 증가세가 다소 둔화되고 재고 증가세는 점차 확대되고 있으며 수출은 

In [6]:
import pandas as pd
from ekonlpy.tag import Mecab
input_file = "./data/minutes/minutes.csv"
df = pd.read_csv(input_file)

df["Economic Situation"] = ""
df["Foreign Currency"] = ""
df["Financial Markets"] = ""
df["Monetary Policy"] = ""
df["Participant Views"] = ""
df["Government View"] = ""

df["Economic Situation count"] = ""
df["Foreign Currency count"] = ""
df["Financial Markets count"] = ""
df["Monetary Policy count"] = ""
df["Participant Views count"] = ""
df["Government View count"] = ""

df["Foreign Currency tokens"] = ""
df["Financial Markets tokens"] = ""

df["Foreign Currency ngram"] = ""
df["Financial Markets ngram"] = ""

for i, row in df.iterrows() :
    sections, section_texts = preprocess_minutes(row['minutes'])
    print(i)
    #print(' '.join(section_texts[4]))
    
    df["Economic Situation"][i] = ' '.join(section_texts[0])
    df["Foreign Currency"][i] = ' '.join(section_texts[1])
    df["Financial Markets"][i] = ' '.join(section_texts[2])
    df["Monetary Policy"][i] = ' '.join(section_texts[3])
    df["Participant Views"][i] = ' '.join(section_texts[4])
    df["Government View"][i] = ' '.join(section_texts[5])

    df["Economic Situation count"][i] = len(section_texts[0])
    df["Foreign Currency count"][i] = len(section_texts[1])
    df["Financial Markets count"][i] = len(section_texts[2])
    df["Monetary Policy count"][i] = len(section_texts[3])
    df["Participant Views count"][i] = len(section_texts[4])
    df["Government View count"][i] = len(section_texts[5])
    #print(1)
    fxtokens,fxngrams = text2ngram(df["Foreign Currency"][i])
    df["Foreign Currency tokens"][i] = ','.join(fxtokens)
    df["Foreign Currency ngram"][i] =  ','.join(fxngrams)
    
    fmtokens,fmngrams = text2ngram(df["Financial Markets"][i])
    df["Financial Markets tokens"][i] = ','.join(fmtokens)
    df["Financial Markets ngram"][i] =  ','.join(fmngrams)
    #print(2)
        
        
df.to_csv("./data/minutes/minutes_preprocessed.csv", index = False)


0


/usr/lib/python3/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/lib/python3/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/lib/python3/dist-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/lib/python3/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: 

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157


In [7]:
import pandas as pd
df = pd.read_csv("./data/minutes/minutes_preprocessed.csv")
df

,date,minutes,Economic Situation,Foreign Currency,Financial Markets,Monetary Policy,Participant Views,Government View,Economic Situation count,Foreign Currency count,Financial Markets count,Monetary Policy count,Participant Views count,Government View count,Foreign Currency tokens,Financial Markets tokens,Foreign Currency ngram,Financial Markets ngram
0,20060810,minute16.hwp금융통화위원회 의사록2006년도 제16차 회의1. 일 자...,NaN,NaN,NaN,NaN,일부 위원은 다음과 같은 점을 종합적으로 고려할 때 이번 달에는 콜금리 - 6 -목...,NaN,0,0,0,0,78,0,NaN,NaN,NaN,NaN
1,20180412,2018년도 제7차금융통화위원회(정기) 의사록한 국 은 행- 1 -1. 일 자 20...,일부 위원은 금년 1/4분기 중 정부의 재정 조기집행에도 불구하고 세입이 더빠르게 ...,일부 위원은 최근 국제금융시장에서 Libor금리가 급등하면서 글로벌 달러자금의 유동...,NaN,NaN,일부 위원은 금년 1/4분기 중 정부의 재정 조기집행에도 불구하고 세입이 더빠르게 ...,NaN,74,69,0,0,400,0,"국제/NNG,금융시장/NNG,금리/NNG,급등/NNG,하/XSV,글로벌/NNG,달러...",NaN,"fed/NNG;금리/NNG;인상/NNG;속도/NNG;가속/NNG,국제/NNG;금융시...",NaN
2,20070608,금융통화위원회 의사록(2007년도 제13차)(2007.6.8).hwp금융통화위원회 ...,NaN,NaN,NaN,NaN,일부 위원은 다음과 같은 점을 종합적으로 고려하여 다음 통화정책방향결정시까지 콜금리...,NaN,0,0,0,0,80,0,NaN,NaN,NaN,NaN
3,20070111,minute2.hwp금융통화위원회 의사록2007년도 제2차 회의1. 일 자 ...,NaN,NaN,NaN,NaN,"일부 위원은 다음과 같은 경기흐름, 물가전망, 금융시장 상황 등을 종합적으로 고려할...",NaN,0,0,0,0,96,0,NaN,NaN,NaN,NaN
4,20051208,130.1.17.56-minute26.hwp- 1 -금융통화위원회 의사록2005년도...,일부 위원은 경제성장률 전망치가 전년동기대비로 보면 내년 하반기에 둔화되는 것으로 ...,NaN,NaN,NaN,일부 위원은 다음과 같은 점을 종합적으로 고려할 때 콜금리 목표를 0.25% 포인트...,NaN,2,0,0,0,84,0,NaN,NaN,NaN,NaN
5,20170223,제4차 금통위 의사록.hwp2017년도 제4차금융통화위원회(정기) 의사록한 국 은 ...,일부 위원은 금년 1월 소비자물가가 전년동월대비 2.0% 상승하는 등 예상보다 빠른...,일부 위원은 Yellen 미 연준 의장이 미국의 금리인상 지연은 바람직하지 않다고 ...,NaN,NaN,일부 위원은 금년 1월 소비자물가가 전년동월대비 2.0% 상승하는 등 예상보다 빠른...,NaN,46,51,0,0,362,0,"금리/NNG,인상/NNG,지연/NNG,바람직/NNG,않/VX,언급/NNG,하/VV,...",NaN,"경제/NNG;성장률/NNG;잠재/NNG;성장률/NNG;상회/NNG,국제/NNG;금융...",NaN
6,20120209,제3차 금통위 의사록.hwp- 1 -금융통화위원회 의사록2012년도 제3차 회의1....,"일부 위원은 최근 국내경제의 성장 모멘텀이 다소 약화되고 있으며, 특히 소비의 경우...",NaN,NaN,NaN,"일부 위원은 최근 국내경제의 성장 모멘텀이 다소 약화되고 있으며, 특히 소비의 경우...",NaN,3,0,0,0,194,0,NaN,NaN,NaN,NaN
7,20121109,- 1 -금융통화위원회 의사록2012년도 제21차 회의1. 일 자 2012년 11월...,"일부 위원은 향후 경기 전망에 대해 물었으며,이에 대해 관련부서에서는 9월 실물지표...",일부 위원은 현재 외환시장에서 일방향의환율절상 기대가 형성되고 있을 가능성에 대한 ...,"일부 위원은 현재 대기업들이 전반적으로는 문제가 없지만, 건설 조선업 등에 속하는 ...",NaN,"일부 위원은 향후 경기 전망에 대해 물었으며,이에 대해 관련부서에서는 9월 실물지표...",NaN,25,12,15,0,318,0,"외환시장/NNG,방향/NNG,환율/NNG,절상/NNG,기대/NNG,형성/NNG,되/...","대기업/NNG,문제/NNG,없/VA,건설/NNG,속하/VV,중견/NNG,대기업/NN...","외화차입/NNG;여건/NNG;양호/NNG,글로벌/NNG;유동성/NNG;증가/NNG,...","경기/NNG;하방/NNG;위험/NNG;증대/NNG,금융시장/NNG;변동성/NNG;완..."
8,20180524,2018년도 제10차금융통화위원회(정기) 의사록한 국 은 행- 1 -1. 일 자 2...,일부 위원은 세계교역량이 신흥국을 중심으로 빠르게 증가하고 있는데 신흥국 중 이를 ...,일부 위원은 최근의 미 달러화 강세가 추가적으로 진행될 경우 일부 취약국에 국한된 ...,NaN,NaN,일부 위원은 세계교역량이 신흥국을 중심으로 빠르게 증가하고 있는데 신흥국 중 이를 ...,NaN,88,73,0,0,384,0,"미/NNG,달러/NNG,강세/NNG,추가/NNG,진행/NNG,되/VV,취약국/NNG...",NaN,"가계/NNG;대출/NNG;관리/NNG;강화/NNG,가계/NNG;대출/NNG;증가/N...",NaN
9,20060112,<C1A632C2F720B1DDC5EBC0A720C0C7BBE7B7CF2E68777...,NaN,NaN,NaN,NaN,"일부 위원은 다음과 같은 점을 종합해 볼 때, 내수회복세가 지속되고 물가와 자산시장...",NaN,0,0,0,0,69,0,NaN,NaN,NaN,NaN


In [88]:
#del df['Foreign Currency ngram']
#del df['Financial Markets ngram']
df.to_csv("./results/minutes_daily_count.csv", index = False)


In [85]:
df

,date,Economic Situation count,Foreign Currency count,Financial Markets count,Monetary Policy count,Participant Views count,Government View count,Foreign Currency ngram,Financial Markets ngram
0,20060810,0,0,0,0,78,0,NaN,NaN
1,20180412,74,69,0,0,400,0,"fed/NNG;금리/NNG;인상/NNG;속도/NNG;가속/NNG,국제/NNG;금융시...",NaN
2,20070608,0,0,0,0,80,0,NaN,NaN
3,20070111,0,0,0,0,96,0,NaN,NaN
4,20051208,2,0,0,0,84,0,NaN,NaN
5,20170223,46,51,0,0,362,0,"경제/NNG;성장률/NNG;잠재/NNG;성장률/NNG;상회/NNG,국제/NNG;금융...",NaN
6,20120209,3,0,0,0,194,0,NaN,NaN
7,20121109,25,12,15,0,318,0,"외화차입/NNG;여건/NNG;양호/NNG,글로벌/NNG;유동성/NNG;증가/NNG,...","경기/NNG;하방/NNG;위험/NNG;증대/NNG,금융시장/NNG;변동성/NNG;완..."
8,20180524,88,73,0,0,384,0,"가계/NNG;대출/NNG;관리/NNG;강화/NNG,가계/NNG;대출/NNG;증가/N...",NaN
9,20060112,0,0,0,0,69,0,NaN,NaN
